In [14]:
from robin_stocks import robinhood as r
import pandas as pd
from colorama import Fore, Style
import numpy as np
import time
import json
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.animation as animation


class TraderClass:
    def __init__(self):
        # read credentials from file
        with open('secrets.json') as f:
            data = json.load(f)
            username = data['username']
            password = data['password']
        self.username = username
        self.password = password
        self.login = r.login(self.username, self.password)
        self.holdings = None
        self.equity = None
        self.buying_power = None
        self.update_account_info()
        self.crypto_symbols = r.crypto.get_crypto_currency_pairs()
        # set up the df for the crypto data
        self.crypto_df = pd.DataFrame(columns=['begins_at', 'open_price', 'close_price', 'high_price', 'low_price', 'volume', 'session', 'interpolated', 'symbol', 'RSI', 'MACD', 'Signal', 'Upper_Band', 'Lower_Band', '%K', '%D', 'MA50', 'MA200', 'Price', 'equity', 'buying_power'])

    #* Account Information Functions
    def update_account_info(self):
        self.holdings = r.account.build_holdings()
        self.equity = r.profiles.load_account_profile(info="equity")
        self.buying_power = r.profiles.load_account_profile(info="buying_power")
        print(Fore.GREEN + "Account information updated." + Style.RESET_ALL)

    #* Technical Indicator Functions
    def calculate_RSI(self, df, window_length=14):
        delta = df['close_price'].diff()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        roll_up1 = up.ewm(span=window_length).mean()
        roll_down1 = down.abs().ewm(span=window_length).mean()
        RS = roll_up1 / roll_down1
        RSI = 100.0 - (100.0 / (1.0 + RS))
        return RSI

    def calculate_MACD(self, df, short_window=12, long_window=26):
        short_ema = df['close_price'].ewm(span=short_window, adjust=False).mean()
        long_ema = df['close_price'].ewm(span=long_window, adjust=False).mean()
        MACD = short_ema - long_ema
        signal = MACD.ewm(span=9, adjust=False).mean()
        return MACD, signal

    def calculate_Bollinger_Bands(self, df, window_length=20, num_of_std=2):
        rolling_mean = df['close_price'].rolling(window=window_length).mean()
        rolling_std = df['close_price'].rolling(window=window_length).std()
        upper_band = rolling_mean + (rolling_std * num_of_std)
        lower_band = rolling_mean - (rolling_std * num_of_std)
        return upper_band, lower_band

    def calculate_Stochastic_Oscillator(self, df, window_length=14):
        low_min = df['low_price'].rolling(window=window_length).min()
        high_max = df['high_price'].rolling(window=window_length).max()
        k_percent = (df['close_price'] - low_min) / (high_max - low_min) * 100
        d_percent = k_percent.rolling(window=3).mean()
        return k_percent, d_percent

    def calculate_Moving_Average(self, df, window_length=50):
        moving_average = df['close_price'].rolling(window=window_length).mean()
        return moving_average

    def calculate_technical_indicators(self, symbol):
        # Fetch historical data
        historical_data = r.crypto.get_crypto_historicals(
            symbol, interval='day', span='year', bounds='24_7', info=None)

        # Convert to DataFrame
        df = pd.DataFrame(historical_data)

        # Calculate technical indicators and convert the df values to numeric before calculating
        df['open_price'] = pd.to_numeric(df['open_price'])
        df['close_price'] = pd.to_numeric(df['close_price'])
        df['high_price'] = pd.to_numeric(df['high_price'])
        df['low_price'] = pd.to_numeric(df['low_price'])
        df['Volume'] = pd.to_numeric(df['volume'])
        df['RSI'] = self.calculate_RSI(df)
        df['MACD'], df['Signal'] = self.calculate_MACD(df)
        df['Upper_Band'], df['Lower_Band'] = self.calculate_Bollinger_Bands(df)
        df['%K'], df['%D'] = self.calculate_Stochastic_Oscillator(df)
        df['MA50'] = self.calculate_Moving_Average(df, window_length=50)
        df['MA200'] = self.calculate_Moving_Average(df, window_length=200)
        df['Price'] = df['close_price']
        # Drop NaN values
        df = df.dropna()

        # Convert values to datetime
        df['begins_at'] = pd.to_datetime(df['begins_at'], format="%Y-%m-%dT%H:%M:%SZ")
        print(Fore.GREEN + "Technical indicators calculated." + Style.RESET_ALL)
        return df

    #* Generating Signals
    def generate_buy_signal(self, df):
        # Initialize buy signal strength
        buy_signal_strength = 0

        # Check RSI for oversold condition
        if df['RSI'].iloc[-1] < 30:
            buy_signal_strength += 1

        # Check MACD for bullish crossover
        if df['MACD'].iloc[-1] > df['Signal'].iloc[-1]:
            buy_signal_strength += 1

        # Check Bollinger Bands for price breakout above the upper band
        if df['Price'].iloc[-1] > df['Upper_Band'].iloc[-1]:
            buy_signal_strength += 1

        # Check Stochastic Oscillator for bullish crossover
        if df['%K'].iloc[-1] > df['%D'].iloc[-1]:
            buy_signal_strength += 1

        # Check Moving Average for golden cross
        if df['MA50'].iloc[-1] > df['MA200'].iloc[-1]:
            buy_signal_strength += 1

        # Check Volume for high trading volume
        if df['Volume'].iloc[-1] > df['Volume'].rolling(window=10).mean().iloc[-1]:
            buy_signal_strength += 1

        # Check Price for higher lows and higher highs
        if df['Price'].iloc[-1] > df['Price'].iloc[-2] and df['Price'].iloc[-2] > df['Price'].iloc[-3]:
            buy_signal_strength += 1

        # Put equity in the df 
        df['equity'] = self.equity

        print(Fore.GREEN + "Buy signal generated with strength: " + str(buy_signal_strength) + Style.RESET_ALL)
        return buy_signal_strength

    def generate_sell_signal(self, df):
        # Initialize sell signal strength
        sell_signal_strength = 0

        # Check RSI for overbought condition
        if df['RSI'].iloc[-1] > 70:
            sell_signal_strength += 1

        # Check MACD for bearish crossover
        if df['MACD'].iloc[-1] < df['Signal'].iloc[-1]:
            sell_signal_strength += 1

        # Check Bollinger Bands for price breakout below the lower band
        if df['Price'].iloc[-1] < df['Lower_Band'].iloc[-1]:
            sell_signal_strength += 1

        # Check Stochastic Oscillator for bearish crossover
        if df['%K'].iloc[-1] < df['%D'].iloc[-1]:
            sell_signal_strength += 1

        # Check Moving Average for death cross
        if df['MA50'].iloc[-1] < df['MA200'].iloc[-1]:
            sell_signal_strength += 1

        # Check Volume for high trading volume
        if df['Volume'].iloc[-1] > df['Volume'].rolling(window=10).mean().iloc[-1]:
            sell_signal_strength += 1

        # Check Price for lower highs and lower lows
        if df['Price'].iloc[-1] < df['Price'].iloc[-2] and df['Price'].iloc[-2] < df['Price'].iloc[-3]:
            sell_signal_strength += 1

        print(Fore.GREEN + "Sell signal generated with strength: " + str(sell_signal_strength) + Style.RESET_ALL)
        return sell_signal_strength

    #* Order Execution Functions
    def execute_buy_order(self, symbol, buy_signal_strength):
        # Calculate the amount to buy
        amount_to_buy = max(1.0, 0.01 * float(self.buying_power))

        # Execute the buy order
        r.order_buy_crypto_by_price(symbol, amount_to_buy)

        print(Fore.GREEN + "Buy order executed for " + symbol + " with amount: $" + str(amount_to_buy) + Style.RESET_ALL)

    def execute_sell_order(self, symbol, sell_signal_strength):
        # Calculate the amount to sell
        amount_to_sell = sell_signal_strength * float(self.holdings[symbol]['quantity'])

        # Execute the sell order
        r.order_sell_crypto_by_quantity(symbol, amount_to_sell)

        print(Fore.GREEN + "Sell order executed for " + symbol + " with quantity: " + str(amount_to_sell) + Style.RESET_ALL)

    #* Main Loop
    def main_loop(self, symbol):
        while True:
            # Calculate technical indicators
            df = self.calculate_technical_indicators(symbol)

            # Generate buy and sell signals
            buy_signal_strength = self.generate_buy_signal(df)
            sell_signal_strength = self.generate_sell_signal(df)

            # Execute buy order if buy signal strength is greater than 2
            if buy_signal_strength > 2:
                print(Fore.GREEN + "Buy signal strength is greater than 2." + Style.RESET_ALL)
                self.execute_buy_order(symbol, buy_signal_strength)

            # Execute sell order if sell signal strength is greater than 2
            if sell_signal_strength > 2:
                print(Fore.YELLOW + "Sell signal strength is greater than 2." + Style.RESET_ALL)
                self.execute_sell_order(symbol, sell_signal_strength)

            # print what we're holding, the technical indicators as a natural language paragraph, and the buy and sell signals
            print(Fore.BLUE + "Holding: " + str(self.holdings) + Style.RESET_ALL)
        

            # Update account information
            self.update_account_info()

            # Sleep for 5 minutes
            time.sleep(60*5)




In [15]:

trader = TraderClass()
# run for 1 hour on BTC
trader.main_loop('BTC')


Error: The keyword "equity" is not a key in the dictionary.
Account information updated.
Technical indicators calculated.
Buy signal generated with strength: 1
Sell signal generated with strength: 2
Error: The keyword "equity" is not a key in the dictionary.
Account information updated.


KeyboardInterrupt: 